In [1]:
# import required functions
import os.path
import tensorflow as tf
import helper
import warnings
import cv2
import numpy as np
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
# Define Variables
Reg = 0.001

In [3]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [4]:
# Load VGG 
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    # Load the model & weight 
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3, layer4, layer7

In [5]:
# Define the layers
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    conv_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='SAME', 
                                       kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    deconv_layer7 = tf.layers.conv2d_transpose(conv_1x1_layer7, num_classes, 4, 2, padding='SAME', 
                                               kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    conv_1x1_layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='SAME',
                                       kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    skip_connect1 = tf.add (deconv_layer7, conv_1x1_layer4)
    deconv_skip_connect1 = tf.layers.conv2d_transpose(skip_connect1, num_classes, 4, 2, padding='SAME', 
                                               kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    conv_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='SAME', 
                                       kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    skip_connect2 = tf.add (deconv_skip_connect1, conv_1x1_layer3)
    output = tf.layers.conv2d_transpose(skip_connect2, num_classes, 16, 8, padding='SAME', 
                                        kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg)) 
    
    return output

In [6]:
# Optimizer
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1,num_classes))
    labels = tf.reshape(correct_label, (-1,num_classes))
    cross_entropy_loss = tf.reduce_mean (tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss

In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    #Train function 
    for epoch in range(epochs):
        print("EPOCH: {}...".format(epoch+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict = {input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0001})
        print("LOSS: {}".format(loss))

In [8]:
def run():
    num_classes = 5
    image_shape = (256, 512) #1/4 Resize
    data_dir = './data'
    runs_dir = './runs'
    #tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        epochs = 35
        batch_size = 16
        image_input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        output = layers(layer3, layer4, layer7, num_classes)
        #TF Placeholder for correct_label,learning_rate
        correct_label = tf.placeholder(dtype = tf.int32, shape = [None, None, None, num_classes])
        learning_rate = tf.placeholder(dtype = tf.float32)
        logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, image_input, correct_label, keep_prob, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        #helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)
        video_file = 'project_video.mp4'
        helper.gen_video_output(sess, logits, keep_prob, image_input, video_file, image_shape)
        print ("Finished..")

In [9]:
run()

INFO:tensorflow:Restoring parameters from b'./data\\vgg\\variables\\variables'
EPOCH: 1...
LOSS: 0.354019433259964
EPOCH: 2...
LOSS: 0.09376213699579239
EPOCH: 3...
LOSS: 0.10658495128154755
EPOCH: 4...
LOSS: 0.14598815143108368
EPOCH: 5...
LOSS: 0.07171033322811127
EPOCH: 6...
LOSS: 0.12964798510074615
EPOCH: 7...
LOSS: 0.17696329951286316
EPOCH: 8...
LOSS: 0.049568790942430496
EPOCH: 9...
LOSS: 0.11057426780462265
EPOCH: 10...
LOSS: 0.05973811075091362
EPOCH: 11...
LOSS: 0.08218207210302353
EPOCH: 12...
LOSS: 0.08818574994802475
EPOCH: 13...
LOSS: 0.04626431688666344
EPOCH: 14...
LOSS: 0.06816785782575607
EPOCH: 15...
LOSS: 0.08496271818876266
EPOCH: 16...
LOSS: 0.034095507115125656
EPOCH: 17...
LOSS: 0.06372690945863724
EPOCH: 18...
LOSS: 0.05004159361124039
EPOCH: 19...
LOSS: 0.05129368230700493
EPOCH: 20...
LOSS: 0.052184827625751495
EPOCH: 21...
LOSS: 0.1304827779531479
EPOCH: 22...
LOSS: 0.06904139369726181
EPOCH: 23...
LOSS: 0.04709292948246002
EPOCH: 24...
LOSS: 0.035840306431